In [1]:
import sqlite3 as sql
import pandas as pd

try:
    conn = sql.connect('population_data.db')
    # print(conn)
except sql.Error as err:
    print('ERROR!' + err)

In [2]:
pd.read_sql(
    """
        select name
        from sqlite_master
        where type='table';
    """
    , conn)

,name
0,sqlite_sequence
1,facts


In [3]:
pd.read_sql(
    """
        select *
        from facts
        ;
    """
    , conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230.0,652230.0,0.0,3.256434e+07,2.32,38.57,13.89,1.51
1,2,al,Albania,28748.0,27398.0,1350.0,3.029278e+06,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741.0,2381741.0,0.0,3.954217e+07,1.84,23.67,4.31,0.92
3,4,an,Andorra,468.0,468.0,0.0,8.558000e+04,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700.0,1246700.0,0.0,1.962535e+07,2.78,38.78,11.49,0.46
...,...,...,...,...,...,...,...,...,...,...,...
256,257,zh,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,258,xo,Indian Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,259,zn,Pacific Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,260,oo,Southern Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
pd.read_sql(
    """
        select facts.name , facts.population as population
        from facts
        join(
            select max(population) as population from facts where name != 'World'
        ) as result
        where name != 'World' and facts.population = result.population
        ;
    """
    , conn)


,name,population
0,China,1367485388


In [5]:
pd.read_sql(
    """
        select facts.name , facts.population as population
        from facts
        join(
            select min(population) as population from facts where name != 'World'
        ) as result
        where name != 'World' and facts.population = result.population
        ;
    """
    , conn)

,name,population
0,Antarctica,0


In [6]:
pd.read_sql(
    """
        select facts.name , facts.population_growth as population_growth
        from facts
        join(
            select max(population_growth) as population_growth from facts
        ) as result
        where name != 'World' and facts.population_growth = result.population_growth
        ;
    """
    , conn)

,name,population_growth
0,South Sudan,4.02


In [7]:
pd.read_sql(
    """
        select facts.name , facts.population_growth as population_growth
        from facts
        join(
            select min(population_growth) as population_growth from facts
        ) as result
        where facts.name != 'World' and facts.population_growth = result.population_growth
        ;
    """
    , conn)

,name,population_growth
0,Holy See (Vatican City),0.0
1,Cocos (Keeling) Islands,0.0
2,Greenland,0.0
3,Pitcairn Islands,0.0


In [8]:
pd.read_sql(
    """
        select facts.name , facts.population/facts.area as population_density
        from facts
        join(
            select max(population/area) as population_density from facts
        ) as result
        where name != 'World' and (facts.population/facts.area)=result.population_density
        ;
    """
    , conn)


,name,population_density
0,Macau,21168


In [9]:
pd.read_sql(
    """
        select facts.name , facts.population/facts.area as population_density
        from facts
        join(
            select min(population/area) as population_density from facts
        ) as result
        where name != 'World' and (facts.population/facts.area)=result.population_density
        ;
    """
    , conn)

,name,population_density
0,Greenland,0
1,Svalbard,0
2,Falkland Islands (Islas Malvinas),0


In [10]:
pd.read_sql(
    """
        select name
        from facts
        where name != 'World'
        order by migration_rate desc
        limit 10
        ;
    """
    , conn)

,name
0,Qatar
1,American Samoa
2,"Micronesia, Federated States of"
3,Syria
4,Tonga
5,British Virgin Islands
6,Luxembourg
7,Cayman Islands
8,Singapore
9,Nauru


In [11]:
pd.read_sql(
    """
        select name
        from facts
        where name != 'World'
        order by (birth_rate / death_rate) desc
        limit 5
        ;
    """
    , conn)

,name
0,Gaza Strip
1,Kuwait
2,Iraq
3,United Arab Emirates
4,Oman


In [12]:
pd.read_sql(
    """
        select
        (
            sum(population_growth * death_rate)
            - ((sum(population_growth) * sum(death_rate))/count(*))
        )
        /
        (
            sqrt(
                sum(population_growth * population_growth)
                - ((sum(population_growth) * sum(population_growth)) / count(*))
            )
            *
            sqrt(
                sum(death_rate * death_rate) - ((sum(death_rate) * sum(death_rate)) / count(*))
            )
        )
         as pc from facts;
    """
    , conn)

,pc
0,0.234608
